The Calamity roadmap will involve. 

* inverting the wedge
* performing abscal with no sky-model
* calibrating out redundancies.

In this notebook, we generate some simulations to test our methods on.

In [1]:
%matplotlib inline
import numpy as np
import healvis as hv
import healpy as hp
import matplotlib.pyplot as plt
from healvis.data import DATA_PATH
import os
from pyuvdata import UVData

In [2]:
import itertools

In [8]:
#create layout
spacings = [0, 1, 6, 10, 23, 26, 34, 41, 53, 55]
#spacings = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
ddish = 4.0
antpos = [np.array([ddish * m, 0.0, 0.0]) for m in spacings]

In [15]:
# create a noise-like sky
freqs =  np.arange(0,400)*60e6/400 + 120e6 
np.random.seed(0)
Nside = 128
Npix = hp.nside2npix(Nside)
eor = hv.sky_model.construct_skymodel("flat_spec", freqs=freqs, 
                                      Nside=Nside, ref_chan=0, 
                                      sigma=1e-3)
# create a PyGSM sky
gsm = hv.sky_model.construct_skymodel('gsm', 
                                      freqs=freqs, Nside=Nside, 
                                      ref_chan=0)

In [ ]:
import copy

In [ ]:
eor.Nfreqs

In [8]:
sky=copy.deepcopy(gsm)
sky.data += eor.data
skymod_file = f'skymodel_Nside{Nside}.hdf5'
sky.write_hdf5(skymod_file)

...skymodel_Nside128.hdf5 exists and clobber == False, skipping


In [17]:
import copy

In [18]:
sky=copy.deepcopy(gsm)
#sky.data += eor.data
skymod_file_gsm = f'skymodel_Nside{Nside}_gsm.hdf5'
sky.write_hdf5(skymod_file_gsm)

...writing skymodel_Nside128_gsm.hdf5


In [19]:
sky=copy.deepcopy(eor)
#sky.data += eor.data
skymod_file_eor = f'skymodel_Nside{Nside}_eor.hdf5'
sky.write_hdf5(skymod_file_eor)

...writing skymodel_Nside128_eor.hdf5


In [9]:
layout_file=f'array_layout_{len(antpos)}_golomb.txt'
with open(layout_file, 'w') as f:
    f.write('Name\tNumber\tBeamID\tE\tN\tU\n')
    for rownum, row in enumerate(antpos):
        print
        line=f'Ant{rownum}\t{rownum}\t0\t{row[0]}\t{row[1]}\t{row[2]}\n'
        f.write(line)
    f.close()

In [20]:
uvd = hv.simulator.setup_uvdata(array_layout=layout_file,
                                telescope_location=(-30.72152777777791, 21.428305555555557, 1073.0000000093132),
                                telescope_name="HERA", Ntimes=1, time_cadence=16.0, start_time=2458000.1,
                                pols=['xx'], run_check=True, 
                                Nfreqs=400, start_freq=120e6, bandwidth=60e6, make_full=True)
uvh5_data = f'simulated_array_ddish_{int(ddish)}_nant_{len(antpos)}_eor.uvh5'
uvd.write_uvh5(uvh5_data, clobber=True)
beam = hv.beam_model.AnalyticBeam('airy', diameter=ddish)
for chunk in range(4):
    freq_chans=np.arange(chunk * 100, (chunk+1) * 100).astype(int)
    hv.simulator.run_simulation_partial_freq(freq_chans, uvh5_data, 
                                             skymod_file_eor, fov=180, 
                                             beam=beam)

Nbls: 45
...reading skymodel_Nside128_eor.hdf5


antenna_diameters is not set. Using known values for HERA.


Finished: 1, Elapsed 1.20min, Remain 0.000hour, MaxRSS 20.724336GB
...writing to simulated_array_ddish_4_nant_10_eor.uvh5
...reading skymodel_Nside128_eor.hdf5
Finished: 1, Elapsed 1.18min, Remain 0.000hour, MaxRSS 20.724504GB
...writing to simulated_array_ddish_4_nant_10_eor.uvh5
...reading skymodel_Nside128_eor.hdf5
Finished: 1, Elapsed 1.17min, Remain 0.000hour, MaxRSS 20.724564GB
...writing to simulated_array_ddish_4_nant_10_eor.uvh5
...reading skymodel_Nside128_eor.hdf5
Finished: 1, Elapsed 1.18min, Remain 0.000hour, MaxRSS 20.724588GB
...writing to simulated_array_ddish_4_nant_10_eor.uvh5


In [21]:
uvd = hv.simulator.setup_uvdata(array_layout=layout_file,
                                telescope_location=(-30.72152777777791, 21.428305555555557, 1073.0000000093132),
                                telescope_name="HERA", Ntimes=1, time_cadence=16.0, start_time=2458000.1,
                                pols=['xx'], run_check=True, 
                                Nfreqs=400, start_freq=120e6, bandwidth=60e6, make_full=True)
uvh5_data = f'simulated_array_ddish_{int(ddish)}_nant_{len(antpos)}_gsm.uvh5'
uvd.write_uvh5(uvh5_data, clobber=True)
beam = hv.beam_model.AnalyticBeam('airy', diameter=ddish)
for chunk in range(4):
    freq_chans=np.arange(chunk * 100, (chunk+1) * 100).astype(int)
    hv.simulator.run_simulation_partial_freq(freq_chans, uvh5_data, 
                                             skymod_file_gsm, fov=180, 
                                             beam=beam)

Nbls: 45
...reading skymodel_Nside128_gsm.hdf5


antenna_diameters is not set. Using known values for HERA.


Finished: 1, Elapsed 1.16min, Remain 0.000hour, MaxRSS 20.724604GB
...writing to simulated_array_ddish_4_nant_10_gsm.uvh5
...reading skymodel_Nside128_gsm.hdf5
Finished: 1, Elapsed 1.16min, Remain 0.000hour, MaxRSS 20.724648GB
...writing to simulated_array_ddish_4_nant_10_gsm.uvh5
...reading skymodel_Nside128_gsm.hdf5
Finished: 1, Elapsed 1.03min, Remain 0.000hour, MaxRSS 20.72466GB
...writing to simulated_array_ddish_4_nant_10_gsm.uvh5
...reading skymodel_Nside128_gsm.hdf5
Finished: 1, Elapsed 1.04min, Remain 0.000hour, MaxRSS 20.72468GB
...writing to simulated_array_ddish_4_nant_10_gsm.uvh5
